In [29]:
from random import randint, random, sample
##
# Define constants
n_days = 5  # Number of days in a week
n_slots_per_day = 4  # Number of time slots per day
n_rooms = 5  # Number of rooms
n_sections = 10  # Number of sections
n_professors = 5  # Number of professors
n_courses = 8  # Number of courses

# Define room capacities
room_capacities = {'classroom': 60, 'large_hall': 120}

# Define course durations
course_durations = {'theory': 80, 'lab': 180}

# Define a timetable as a 3D array: [day][time_slot][room]
timetable = [[[None for _ in range(n_rooms)] for _ in range(n_slots_per_day)] for _ in range(n_days)]

# Update crossover function to consider classroom availability
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if random() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        # Ensure classroom availability for children
        c1 = ensure_classroom_availability(c1)
        c2 = ensure_classroom_availability(c2)
    return [c1, c2]

# Update mutation function to consider classroom availability
def mutation(timetable, r_mut):
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                # check for a mutation
                if random() < r_mut:
                    # flip the bit
                    if timetable[day][slot][room] is not None:
                        timetable[day][slot][room] = None
    # Ensure classroom availability after mutation
    return ensure_classroom_availability(timetable)

# Function to ensure classroom availability
def ensure_classroom_availability(timetable):
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                if timetable[day][slot][room] is not None:
                    # Check if the room is free
                    if any(timetable[day][slot][r] == timetable[day][slot][room] for r in range(n_rooms) if r != room):
                        # If not free, find a free room
                        for new_room in range(n_rooms):
                            if timetable[day][slot][new_room] is None:
                                timetable[day][slot][new_room] = timetable[day][slot][room]
                                timetable[day][slot][room] = None
                                break
    return timetable

# Function to evaluate the timetable
def evaluate_timetable(timetable):
    clashes = 0
    professor_courses_count = {professor: 0 for professor in range(n_professors)}
    section_courses_count = {section: 0 for section in range(n_sections)}
    section_room_timeslots = {section: set() for section in range(n_sections)}
    room_section_timeslots = {room: set() for room in range(n_rooms)}
    section_classroom = {section: None for section in range(n_sections)}
    
    # Define time slots for morning and afternoon sessions
    morning_slots = range(0, n_slots_per_day // 2)  # Morning slots
    afternoon_slots = range(n_slots_per_day // 2, n_slots_per_day)  # Afternoon slots
    
    for day in range(n_days):
        for slot in morning_slots:  # Morning session
            for room in range(n_rooms):
                section = timetable[day][slot][room]
                if section is not None:
                    # Constraint 1: Check if the section is assigned to another room at the same time
                    if (day, slot) in section_room_timeslots[section]:
                        clashes += 1
                    else:
                        section_room_timeslots[section].add((day, slot))
                    # Constraint 2: Check if the room is assigned to another section at the same time
                    if (day, slot) in room_section_timeslots[room]:
                        clashes += 1
                    else:
                        room_section_timeslots[room].add((day, slot))
                    # Constraint 8: A class should be held in the same classroom across the whole week
                    if section_classroom[section] is not None and section_classroom[section] != room:
                        clashes += 0.2  # Increase clashes by a small amount as this is a soft constraint
                    else:
                        section_classroom[section] = room
                    # Count courses per professor and per section
                    professor = timetable[day][slot][room]
                    if professor in professor_courses_count:  # Check if professor is a valid key
                        professor_courses_count[professor] += 1
                    else:
                        print(f"Invalid professor: {professor}")
                        clashes += 1  # Increment clashes if professor is invalid
                    section_courses_count[section] += 1

        for slot in afternoon_slots:  # Afternoon session
            for room in range(n_rooms):
                section = timetable[day][slot][room]
                if section is not None:
                    # Constraint 1: Check if the section is assigned to another room at the same time
                    if (day, slot) in section_room_timeslots[section]:
                        clashes += 1
                    else:
                        section_room_timeslots[section].add((day, slot))
                    # Constraint 2: Check if the room is assigned to another section at the same time
                    if (day, slot) in room_section_timeslots[room]:
                        clashes += 1
                    else:
                        room_section_timeslots[room].add((day, slot))
                    # Constraint 8: A class should be held in the same classroom across the whole week
                    if section_classroom[section] is not None and section_classroom[section] != room:
                        clashes += 0.2  # Increase clashes by a small amount as this is a soft constraint
                    else:
                        section_classroom[section] = room
                    # Count courses per professor and per section
                    professor = timetable[day][slot][room]
                    if professor in professor_courses_count:  # Check if professor is a valid key
                        professor_courses_count[professor] += 1
                    else:
                        print(f"Invalid professor: {professor}")
                        clashes += 1  # Increment clashes if professor is invalid
                    section_courses_count[section] += 1
    
    # Constraint 3: Check if any professor teaches more than 3 courses
    for professor, count in professor_courses_count.items():
        if count > 3:
            clashes += 1
    
    # Constraint 4: Check if any section has more than 5 courses
    for section, count in section_courses_count.items():
        if count > 5:
            clashes += 1
    
    # Constraint 5: Each course would have two lectures per week not on the same or adjacent days
    for section in range(n_sections):
        lecture_days = []
        for day in range(n_days):
            for slot in morning_slots:  # Only consider morning slots for lecture days
                if timetable[day][slot][room] == section:
                    lecture_days.append(day)
                    break  # Only need to check one slot per day
        # Check if lectures are on the same or adjacent days
        if len(lecture_days) != len(set(lecture_days)) or any(abs(lecture_days[i] - lecture_days[i+1]) == 1 for i in range(len(lecture_days)-1)):
            clashes += 1
    
    # Constraint 6: Lab lectures should be conducted in two consecutive slots
    for section in range(n_sections):
        for day in range(n_days):
            for slot in morning_slots:  # Only consider morning slots for lab sessions
                if timetable[day][slot][room] == section and timetable[day][slot+1][room] == section:
                    if slot % 2 != 0:
                        clashes += 1
    
    # Constraint 7: 15 mins breaks allowed between consecutive classes
    for day in range(n_days):
        for room in range(n_rooms):
            for slot in morning_slots:  # Only consider morning slots for breaks
                if timetable[day][slot][room] is not None and timetable[day][slot+1][room] is not None:
                    if slot % 2 == 0:  # Check if the slot is not a lab slot
                        if slot != 5:  # Ensure we don't go out of bounds
                            if timetable[day][slot+2][room] is not None:
                                clashes += 1  # There should be a break if a class is followed by another class
                        else:
                            clashes += 1  # Last slot of the morning session, no classes can follow
    
    # Soft Constraint 1: Prioritize scheduling theory classes in the morning slots and lab sessions in afternoon slots
    # Soft Constraint 2: Attempt to schedule classes on the same floor whenever possible
    # This is a complex constraint and may not be fully optimized, so we'll apply a small penalty for violations
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                section = timetable[day][slot][room]
                if section is not None:
                    # Determine the floor of the room
                    floor = room // (n_rooms // 2)
                    # Check other rooms on the same floor
                    for other_room in range(floor * (n_rooms // 2), min((floor + 1) * (n_rooms // 2), n_rooms)):
                        if other_room != room and timetable[day][slot][other_room] is not None:
                            clashes += 0.2  # Increase clashes by a small amount as this is a soft constraint
    
    return clashes



####

def encode_chromosome(timetable):
    chromosome = []
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                section = timetable[day][slot][room]
                if section is not None:
                    # Encode course (section) in binary format
                    course_bin = bin(section)[2:].zfill(6)  # Assuming maximum of 64 courses
                    # Encode Theory/Lab (0 for theory, 1 for lab)
                    is_lab_bin = bin(is_lab(section))[2:].zfill(1)
                    # Encode Section
                    section_bin = bin(section % n_sections)[2:].zfill(4)  # Assuming maximum of 16 sections per course
                    # Encode Section-Strength (assuming maximum section strength of 100)
                    section_strength_bin = bin(section_strength(section))[2:].zfill(7)
                    # Encode Professor (assuming maximum of 32 professors)
                    professor_bin = bin(professor(section))[2:].zfill(5)
                    # Encode First-lecture-day (3 bits for 5 days)
                    first_lecture_day_bin = bin(day)[2:].zfill(3)
                    # Encode First-lecture-timeslot (4 bits for 9 slots)
                    first_lecture_slot_bin = bin(slot)[2:].zfill(4)
                    # Encode First-lecture-room (5 bits for 32 rooms)
                    first_lecture_room_bin = bin(room)[2:].zfill(5)
                    # Encode First-lecture-room-size (assuming maximum room size of 100)
                    first_lecture_room_size_bin = bin(room_size(room))[2:].zfill(7)
                    # Encode Second-lecture-day (3 bits for 5 days)
                    second_lecture_day_bin = bin(day)[2:].zfill(3)
                    # Encode Second-lecture-timeslot (4 bits for 9 slots)
                    second_lecture_slot_bin = bin(slot)[2:].zfill(4)
                    # Encode Second-lecture-room (5 bits for 32 rooms)
                    second_lecture_room_bin = bin(room)[2:].zfill(5)
                    # Encode Second-lecture-room-size (assuming maximum room size of 100)
                    second_lecture_room_size_bin = bin(room_size(room))[2:].zfill(7)
                    
                    # Concatenate binary strings to form chromosome
                    chromosome.append(course_bin + is_lab_bin + section_bin + section_strength_bin +
                                       professor_bin + first_lecture_day_bin + first_lecture_slot_bin +
                                       first_lecture_room_bin + first_lecture_room_size_bin +
                                       second_lecture_day_bin + second_lecture_slot_bin +
                                       second_lecture_room_bin + second_lecture_room_size_bin)
    return ''.join(chromosome)

def decode_chromosome(chromosome):
    timetable = [[[None for _ in range(n_rooms)] for _ in range(n_slots_per_day)] for _ in range(n_days)]
    for i in range(0, len(chromosome), 61):  # Assuming each chromosome length is 61 bits
        course = int(chromosome[i:i+6], 2)
        is_lab = int(chromosome[i+6], 2)
        section = int(chromosome[i+7:i+11], 2)
        section_strength = int(chromosome[i+11:i+18], 2)
        professor = int(chromosome[i+18:i+23], 2)
        first_lecture_day = int(chromosome[i+23:i+26], 2)
        first_lecture_slot = int(chromosome[i+26:i+30], 2)
        first_lecture_room = int(chromosome[i+30:i+35], 2)
        first_lecture_room_size = int(chromosome[i+35:i+42], 2)
        second_lecture_day = int(chromosome[i+42:i+45], 2)
        second_lecture_slot = int(chromosome[i+45:i+49], 2)
        second_lecture_room = int(chromosome[i+49:i+54], 2)
        second_lecture_room_size = int(chromosome[i+54:i+61], 2)
        # Decode and assign values to timetable
        timetable[first_lecture_day][first_lecture_slot][first_lecture_room] = course
        timetable[second_lecture_day][second_lecture_slot][second_lecture_room] = course
        # Assuming the rest of the information is used elsewhere in the evaluation function
    return timetable

def fitness_function(timetable):
    conflicts = evaluate_timetable(timetable)  # Calculate total conflicts
    return -conflicts  # Return negative of conflicts as fitness

####








# Function to generate an initial timetable
def generate_initial_timetable():
    timetable = [[[None for _ in range(n_rooms)] for _ in range(n_slots_per_day)] for _ in range(n_days)]
    for day in range(n_days):
        for slot in range(n_slots_per_day):
            for room in range(n_rooms):
                # Randomly assign classes to rooms
                if randint(0, 1) == 1:
                    timetable[day][slot][room] = randint(0, n_sections-1)  # Assign section
    return timetable

# Define genetic algorithm function
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [generate_initial_timetable() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = None, float('inf')
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best score: %.3f" % (gen, best_eval))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = []
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# Define selection function (tournament selection)
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(0, len(pop)-1)
    for ix in sample(range(len(pop)), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# Run genetic algorithm
n_bits = n_days * n_slots_per_day * n_rooms
n_iter = 100
n_pop = 50
r_cross = 0.9
r_mut = 1.0 / n_bits

best_solution, best_score = genetic_algorithm(evaluate_timetable, n_bits, n_iter, n_pop, r_cross, r_mut)
print("Best Solution:", best_solution)
print("Best Score:", best_score)


Invalid professor: 7
Invalid professor: 9
Invalid professor: 9
Invalid professor: 9
Invalid professor: 8
Invalid professor: 8
Invalid professor: 6
Invalid professor: 7
Invalid professor: 6
Invalid professor: 6
Invalid professor: 7
Invalid professor: 9
Invalid professor: 5
Invalid professor: 7
Invalid professor: 9
Invalid professor: 5
Invalid professor: 9
Invalid professor: 9
Invalid professor: 7
Invalid professor: 5
Invalid professor: 6
Invalid professor: 9
Invalid professor: 9
Invalid professor: 5
Invalid professor: 6
Invalid professor: 9
Invalid professor: 8
Invalid professor: 7
Invalid professor: 9
Invalid professor: 5
Invalid professor: 8
Invalid professor: 5
Invalid professor: 7
Invalid professor: 5
Invalid professor: 6
Invalid professor: 6
Invalid professor: 5
Invalid professor: 9
Invalid professor: 7
Invalid professor: 7
Invalid professor: 5
Invalid professor: 7
Invalid professor: 5
Invalid professor: 9
Invalid professor: 7
Invalid professor: 9
Invalid professor: 7
Invalid profe